# Example Ising Model

This is from the following video: https://youtube.com/watch?v=K--1hlv9yv0&feature=share

Hopefully, this will provide a useful guide for us moving forward.

In [2]:
pip install SciencePlots

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(['science','notebook', 'grid'])
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure

OSError: 'science' not found in the style library and input is not a valid URL or path; see `style.available` for list of available styles

In [ ]:
# we will make a 50x50 grid
N = 50

In [ ]:
# +1 is spin up
# - 1 is spin down
init_random = np.random.random((N,N))
lattice_n = np.zeros((N, N))
lattice_n[init_random>=0.75] = 1
lattice_n[init_random<0.75] = -1

# init_random = np.random.random((N,N))
# lattice_p = np.zeros((N, N))
# lattice_p[init_random>=0.25] = 1
# lattice_p[init_random<0.25] = -1

In [ ]:
plt.imshow(lattice_n)